### Se van a normalizar los datos y a reordenar las filas

In [10]:
import pandas as pd

In [11]:
# Diccionario de prueba
a = {
    'col1': [1,2,3,4,5,6,7,8,9,0],
    'col2': [4,5,6,5,6,7,8,2,3,4]
}

df = pd.DataFrame(a)
df

,col1,col2
0,1,4
1,2,5
2,3,6
3,4,5
4,5,6
5,6,7
6,7,8
7,8,2
8,9,3
9,0,4


In [12]:
columnas = []
profundidades = ['_0','_5','_10']

# añado etiqueta de profundidad
for i in range(3):
    columnas_aux = df.columns + profundidades[i]
    [columnas.append(x) for x in columnas_aux]
    
df_2 = pd.DataFrame(columns=columnas)
df_2

,col1_0,col2_0,col1_5,col2_5,col1_10,col2_10


In [13]:
# concateno datos de la tres profundidades en una sola
for cont in range(int(df.shape[0]/3)):
    inicio = cont*3
    df_aux = df.loc[inicio:inicio+2]
    print(df_aux)
    lista = []

    for fila in df_aux.values.tolist():
        for elem in fila :
            lista.append(elem)
    print(lista)
    df_2.loc[cont] = lista
    

   col1  col2
0     1     4
1     2     5
2     3     6
[1, 4, 2, 5, 3, 6]
   col1  col2
3     4     5
4     5     6
5     6     7
[4, 5, 5, 6, 6, 7]
   col1  col2
6     7     8
7     8     2
8     9     3
[7, 8, 8, 2, 9, 3]


In [14]:
df_2

,col1_0,col2_0,col1_5,col2_5,col1_10,col2_10
0,1,4,2,5,3,6
1,4,5,5,6,6,7
2,7,8,8,2,9,3


# Con estructura avistamientos

In [15]:
# Estructura quitando las columnas de las coordenadas pues no nos sirven
df = pd.read_excel('../4EstructuraFinalSinMissings.xlsx')
df = df.fillna(method='ffill', axis=0).set_index(['Latitud','Longitud','Fecha','Avistamientos','Profundidad']).drop(['level_0','index'],axis = 1)
# Eliminar columnas Coords
cols = [c for c in df.columns if 'C' not in c]
df=df[cols]

df

mlotst  \
Latitud    Longitud   Fecha      Avistamientos Profundidad              
-42.666667 -74.083333 2015-05-27 1.0           0.494025     49.592583   
                                               5.078224     49.592583   
                                               9.572997     49.592583   
-41.833333 -73.833333 2014-06-04 1.0           0.494025     11.139256   
                                               5.078224     11.139256   
...                                                               ...   
-18.500000 -70.333333 2014-08-17 1.0           5.078224     10.528886   
                                               9.572997     10.528886   
-18.416667 -70.333333 2015-08-18 1.0           0.494025     14.496293   
                                               5.078224     14.496293   
                                               9.572997     14.496293   

                                                                 zos  \
Latitud    Longitud   Fecha      Avistamientos Profundidad             
-42.666667 -74.083333 2015-05-27 1.0           0.494025     0.106815   
                                               5.078224     0.106815   
                                               9.572997     0.106815   
-41.833333 -73.833333 2014-06-04 1.0           0.494025     0.058901   
                                               5.078224     0.058901   
...                                                              ...   
-18.500000 -70.333333 2014-08-17 1.0           5.078224     0.036012   
                                               9.572997     0.036012   
-18.416667 -70.333333 2015-08-18 1.0           0.494025     0.054628   
                                               5.078224     0.054628   
                                               9.572997     0.054628   

                                                              bottomT  \
Latitud    Longitud   Fecha      Avistamientos Profundidad              
-42.666667 -74.083333 2015-05-27 1.0           0.494025     12.775384   
                                               5.078224     12.775384   
                                               9.572997     12.775384   
-41.833333 -73.833333 2014-06-04 1.0           0.494025     11.645955   
                                               5.078224     11.645955   
...                                                               ...   
-18.500000 -70.333333 2014-08-17 1.0           5.078224     14.238075   
                                               9.572997     14.238075   
-18.416667 -70.333333 2015-08-18 1.0           0.494025     15.651692   
                                               5.078224     15.651692   
                                               9.572997     15.651692   

                                                               thetao  \
Latitud    Longitud   Fecha      Avistamientos Profundidad              
-42.666667 -74.083333 2015-05-27 1.0           0.494025     12.781243   
                                               5.078224     12.780511   
                                               9.572997     12.780511   
-41.833333 -73.833333 2014-06-04 1.0           0.494025     11.601275   
                                               5.078224     11.628376   
...                                                               ...   
-18.500000 -70.333333 2014-08-17 1.0           5.078224     15.194647   
                                               9.572997     14.896542   
-18.416667 -70.333333 2015-08-18 1.0           0.494025     15.679525   
                                               5.078224     15.642902   
                                               9.572997     15.627522   

                                                                   so  \
Latitud    Longitud   Fecha      Avistamientos Profundidad              
-42.666667 -74.083333 2015-05-27 1.0           0.494025     33.344524   
                                               5.078224  

In [16]:
columnas = []
profundidades = ['_0','_5','_10']

# añado etiqueta de profundidad
def resetea_atributos():
    for i in range(3):
        columnas_aux = df.columns + profundidades[i]
        [columnas.append(x) for x in columnas_aux]

    df_atributos = pd.DataFrame(columns=columnas)
    return df_atributos
# df_atributos

In [17]:
df_atributos = resetea_atributos()
# meto tres profundidades consecutivas
for cont in range(int(df.shape[0]/3)):
    inicio = cont*3
    df_aux = df.iloc[inicio:inicio+3]
    lista = list()

    for fila in df_aux.values.tolist():
        for elem in fila :
            lista.append(elem)
            
    dict_atr = dict(zip(df_atributos.columns,lista))
    df_atr_aux = pd.DataFrame(dict_atr,index=[0])
    df_atributos = pd.concat([df_atributos,df_atr_aux])
    
df_atributos

,mlotst_0,zos_0,bottomT_0,thetao_0,so_0,uo_0,vo_0,mlotst_5,zos_5,bottomT_5,...,so_5,uo_5,vo_5,mlotst_10,zos_10,bottomT_10,thetao_10,so_10,uo_10,vo_10
0,49.592583,0.106815,12.775384,12.781243,33.344524,0.034791,-0.079958,49.592583,0.106815,12.775384,...,33.344524,0.028687,-0.051881,49.592583,0.106815,12.775384,12.780511,33.344524,0.023804,-0.039064
0,11.139256,0.058901,11.645955,11.601275,33.599354,0.113529,0.039064,11.139256,0.058901,11.645955,...,33.623768,0.107425,0.034181,11.139256,0.058901,11.645955,11.745567,33.616138,0.092776,0.020753
0,21.210365,0.119938,9.503555,12.053194,33.436077,0.018311,0.084231,21.210365,0.119938,9.503555,...,33.436077,0.003662,0.059206,21.210365,0.119938,9.503555,12.053926,33.436077,-0.001831,0.046388
0,22.125921,0.056154,12.565905,12.562243,33.347576,-0.005493,0.078738,22.125921,0.056154,12.565905,...,33.347576,-0.029298,0.067141,22.125921,0.056154,12.565905,12.562243,33.347576,-0.042116,0.061647
0,22.125921,0.018921,12.343974,12.338115,33.342999,-0.048830,0.056764,22.125921,0.018921,12.343974,...,33.342999,-0.064699,0.042116,22.125921,0.018921,12.343974,12.338847,33.342999,-0.061037,0.036622
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
0,10.528886,0.219428,15.332346,17.619770,34.888760,-0.071413,0.136113,10.528886,0.219428,15.332346,...,34.910122,-0.062258,0.086062,10.528886,0.219428,15.332346,16.786980,34.946747,-0.037233,0.018921
0,10.528886,0.154729,14.305460,17.476212,34.858242,-0.087283,0.109867,10.528886,0.154729,14.305460,...,34.861294,-0.084231,0.046999,10.528886,0.154729,14.305460,15.794519,34.853664,-0.062868,0.066530
0,10.528886,-0.029603,13.162114,14.737602,34.777367,-0.056764,0.037233,10.528886,-0.029603,13.162114,...,34.778893,-0.044557,-0.008545,10.528886,-0.029603,13.162114,13.714377,34.777367,-0.016480,-0.014039
0,10.528886,0.036012,14.238075,15.366772,34.760582,-0.035401,0.110477,10.528886,0.036012,14.238075,...,34.762108,-0.030519,0.079958,10.528886,0.036012,14.238075,14.896542,34.766685,-0.021363,0.040895


## normalizar

In [18]:
# sin uso de bibliotecas 
df = df_atributos.copy()
normalized_df=(df-df.min())/(df.max()-df.min())
print(df.mlotst_0.max())
normalized_df

147.2518157958984


,mlotst_0,zos_0,bottomT_0,thetao_0,so_0,uo_0,vo_0,mlotst_5,zos_5,bottomT_5,...,so_5,uo_5,vo_5,mlotst_10,zos_10,bottomT_10,thetao_10,so_10,uo_10,vo_10
0,0.306609,0.394561,0.456661,0.154026,0.521541,0.584559,0.177163,0.306609,0.394561,0.456661,...,0.436922,0.576687,0.244929,0.306609,0.394561,0.456661,0.163512,0.387808,0.564677,0.341365
0,0.033586,0.314007,0.371637,0.049150,0.563862,0.742647,0.312111,0.033586,0.314007,0.371637,...,0.491500,0.734969,0.350864,0.033586,0.314007,0.371637,0.072551,0.445525,0.705224,0.420080
0,0.105092,0.416624,0.210355,0.089317,0.536746,0.551471,0.363322,0.105092,0.416624,0.210355,...,0.454816,0.526380,0.381668,0.105092,0.416624,0.210355,0.099652,0.407263,0.512438,0.453815
0,0.111593,0.309389,0.440891,0.134562,0.522048,0.503676,0.357093,0.111593,0.309389,0.440891,...,0.437519,0.460123,0.391435,0.111593,0.309389,0.440891,0.144329,0.388457,0.430348,0.473896
0,0.111593,0.246793,0.424184,0.114641,0.521288,0.416667,0.332180,0.111593,0.246793,0.424184,...,0.436624,0.388957,0.360631,0.111593,0.246793,0.424184,0.124694,0.387484,0.391791,0.440964
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
0,0.029252,0.583889,0.649151,0.584077,0.778003,0.371324,0.422145,0.029252,0.583889,0.649151,...,0.742916,0.393865,0.414726,0.029252,0.583889,0.649151,0.515643,0.728275,0.440298,0.417671
0,0.029252,0.475115,0.571846,0.571317,0.772935,0.339461,0.392388,0.029252,0.475115,0.571846,...,0.733373,0.349693,0.366642,0.029252,0.475115,0.571846,0.428415,0.708496,0.388060,0.480321
0,0.029252,0.165213,0.485774,0.327908,0.759503,0.400735,0.310035,0.029252,0.165213,0.485774,...,0.717268,0.429448,0.298272,0.029252,0.165213,0.485774,0.245590,0.692283,0.482587,0.374297
0,0.029252,0.275526,0.566773,0.383829,0.756716,0.443627,0.393080,0.029252,0.275526,0.566773,...,0.713987,0.457669,0.407213,0.029252,0.275526,0.566773,0.349491,0.690013,0.472637,0.446586


In [19]:
49.592583 / 147.2518157958984

0.336787582088216

In [20]:
n=(1-0)/(2-0)
n=(1--1)/(2--1)
n=(1--1)/(1--1)

n

1.0

In [21]:
# con scikit learn ejemplos
from sklearn import preprocessing
import numpy as np

X = [[ 1., -1.,  2.],
     [ 2.,  0.,  0.],
     [ 0.,  1., -1.]]
X_normalized = preprocessing.normalize(X, norm='l2')

X_normalized



array([[ 0.40824829, -0.40824829,  0.81649658],
       [ 1.        ,  0.        ,  0.        ],
       [ 0.        ,  0.70710678, -0.70710678]])

In [22]:
X = [[ 1., -1.,  2.],
     [ 2.,  0.,  0.],
     [ 0.,  1., -1.]]
X_normalized = preprocessing.normalize(X, norm='l1')

X_normalized

array([[ 0.25, -0.25,  0.5 ],
       [ 1.  ,  0.  ,  0.  ],
       [ 0.  ,  0.5 , -0.5 ]])

In [45]:
# con estructura
X = df_atributos.values.tolist()
X_normalized = preprocessing.normalize(X, norm='l2')

X_normalized

df_norm = pd.DataFrame(X_normalized,columns=df_atributos.columns)
df_norm.to_excel('dfNormalizado.xlsx')